# Facies classification using Machine Learning #
## LA Team Submission 6 ## 
### _[Lukas Mosser](https://at.linkedin.com/in/lukas-mosser-9948b32b/en), [Alfredo De la Fuente](https://pe.linkedin.com/in/alfredodelafuenteb)_ ####

In this approach for solving the facies classfication problem ( https://github.com/seg/2016-ml-contest. ) we will explore the following statregies:
- Features Exploration: based on [Paolo Bestagini's work](https://github.com/seg/2016-ml-contest/blob/master/ispl/facies_classification_try02.ipynb), we will consider imputation, normalization and augmentation routines for the initial features.
- Model tuning: we use TPOT to come up with a good enough pipeline, and then tune the hyperparameters of the model obtained using HYPEROPT.

## Packages and Libraries

In [1]:
%%sh
pip install pandas
pip install scikit-learn
pip install tpot
pip install hyperopt
pip install xgboost

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
#from __future__ import print_function
import numpy as np
#%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold , StratifiedKFold
from classification_utilities import display_cm, display_adj_cm
from sklearn.metrics import confusion_matrix, f1_score
from sklearn import preprocessing, impute
from sklearn.model_selection import LeavePGroupsOut
from sklearn.multiclass import OneVsOneClassifier
from scipy.signal import medfilt

## Data Preprocessing 

We procceed to run [Paolo Bestagini's routine](https://github.com/seg/2016-ml-contest/blob/master/ispl/facies_classification_try02.ipynb) to include a small window of values to acount for the spatial component in the log analysis, as well as the gradient information with respect to depth. This will be our prepared training dataset.

In [3]:
#Load Data
data = pd.read_csv('../facies_vectors.csv')

# Parameters
feature_names = ['GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS']
facies_names = ['SS', 'CSiS', 'FSiS', 'SiSh', 'MS', 'WS', 'D', 'PS', 'BS']
facies_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00', '#1B4F72','#2E86C1', '#AED6F1', '#A569BD', '#196F3D']

# Store features and labels
X = data[feature_names].values 
y = data['Facies'].values 

# Store well labels and depths
well = data['Well Name'].values
depth = data['Depth'].values

# Fill 'PE' missing values with mean
imp = impute.SimpleImputer(strategy='mean')   #Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(X)
X = imp.transform(X)

In [4]:
# Feature windows concatenation function
def augment_features_window(X, N_neig):
    
    # Parameters
    N_row = X.shape[0]
    N_feat = X.shape[1]

    # Zero padding
    X = np.vstack((np.zeros((N_neig, N_feat)), X, (np.zeros((N_neig, N_feat)))))

    # Loop over windows
    X_aug = np.zeros((N_row, N_feat*(2*N_neig+1)))
    for r in np.arange(N_row)+N_neig:
        this_row = []
        for c in np.arange(-N_neig,N_neig+1):
            this_row = np.hstack((this_row, X[r+c]))
        X_aug[r-N_neig] = this_row

    return X_aug


# Feature gradient computation function
def augment_features_gradient(X, depth):
    
    # Compute features gradient
    d_diff = np.diff(depth).reshape((-1, 1))
    d_diff[d_diff==0] = 0.001
    X_diff = np.diff(X, axis=0)
    X_grad = X_diff / d_diff
        
    # Compensate for last missing value
    X_grad = np.concatenate((X_grad, np.zeros((1, X_grad.shape[1]))))
    
    return X_grad


# Feature augmentation function
def augment_features(X, well, depth, N_neig=1):
    
    # Augment features
    X_aug = np.zeros((X.shape[0], X.shape[1]*(N_neig*2+2)))
    for w in np.unique(well):
        w_idx = np.where(well == w)[0]
        X_aug_win = augment_features_window(X[w_idx, :], N_neig)
        X_aug_grad = augment_features_gradient(X[w_idx, :], depth[w_idx])
        X_aug[w_idx, :] = np.concatenate((X_aug_win, X_aug_grad), axis=1)
    
    # Find padded rows
    padded_rows = np.unique(np.where(X_aug[:, 0:7] == np.zeros((1, 7)))[0])
    
    return X_aug, padded_rows

X_aug, padded_rows = augment_features(X, well, depth)

In [5]:
# Initialize model selection methods
lpgo = LeavePGroupsOut(2)

# Generate splits
split_list = []
for train, val in lpgo.split(X, y, groups=data['Well Name']):
    hist_tr = np.histogram(y[train], bins=np.arange(len(facies_names)+1)+.5)
    hist_val = np.histogram(y[val], bins=np.arange(len(facies_names)+1)+.5)
    if np.all(hist_tr[0] != 0) & np.all(hist_val[0] != 0):
        split_list.append({'train':train, 'val':val})
    
        
def preprocess():
    
    # Preprocess data to use in model
    X_train_aux = []
    X_test_aux = []
    y_train_aux = []
    y_test_aux = []
    
    # For each data split
    for split in split_list:
        # Remove padded rows
        split_train_no_pad = np.setdiff1d(split['train'], padded_rows)

        # Select training and validation data from current split
        X_tr = X_aug[split_train_no_pad, :]
        X_v = X_aug[split['val'], :]
        y_tr = y[split_train_no_pad]
        y_v = y[split['val']]

        # Select well labels for validation data
        well_v = well[split['val']]

        # Feature normalization
        scaler = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_tr)
        X_tr = scaler.transform(X_tr)
        X_v = scaler.transform(X_v)

        X_train_aux.append( X_tr )
        X_test_aux.append( X_v )
        y_train_aux.append( y_tr )
        y_test_aux.append (  y_v )

        X_train = np.concatenate( X_train_aux )
        X_test = np.concatenate ( X_test_aux )
        y_train = np.concatenate ( y_train_aux )
        y_test = np.concatenate ( y_test_aux )
    
    return X_train , X_test , y_train , y_test 

X_train, X_test, y_train, y_test = preprocess()
y_train = y_train - 1 
y_test = y_test - 1 

## Data Analysis

In this section we will run a Cross Validation routine 

In [6]:
import xgboost as xgb
from xgboost.sklearn import  XGBClassifier
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.pipeline import make_pipeline

In [7]:
SEED = 314159265
VALID_SIZE = 0.2
TARGET = 'outcome'

# Scoring and optimization functions

def score(params):
    print("Training with params: ")
    print(params)
    #clf = xgb.XGBClassifier(**params) 
    #clf.fit(X_train, y_train)
    #y_predictions = clf.predict(X_test)
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_test, label=y_test)
    watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    gbm_model = xgb.train(params, dtrain, num_round,
                          evals=watchlist,
                          verbose_eval=True)
    if hasattr(gbm_model, 'best_iteration') and gbm_model.best_iteration is not None:
        best_iteration = gbm_model.best_iteration
    else:
        best_iteration = gbm_model.num_boosted_rounds()
    
    print(f"Best iteration: {best_iteration}")
    y_predictions = gbm_model.predict(dvalid,
                                    iteration_range=(0,best_iteration))
    
    score = f1_score (y_test, y_predictions , average ='micro')
    print("\tScore {0}\n\n".format(score))
    loss = 1 - score
    return {'loss': loss, 'status': STATUS_OK}


def optimize(random_state=SEED):
    space = {
        'n_estimators': hp.quniform('n_estimators', 100, 150, 1),
        'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
        'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
        'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
        'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
        'eval_metric': 'mlogloss',
        'objective': 'multi:softmax',
        'nthread': 4,
        'booster': 'gbtree',
        'tree_method': 'exact',
        'silent': 1,
        'num_class' : 9,
        'seed': random_state
    }
    # Use the fmin function from Hyperopt to find the best hyperparameters
    best = fmin(score, space, algo=tpe.suggest,  max_evals=5)
    return best

In [8]:
best_hyperparams = optimize()
print("The best hyperparameters are: ", "\n")
print(best_hyperparams)

Training with params:                                
{'booster': 'gbtree', 'colsample_bytree': 0.75, 'eta': 0.47500000000000003, 'eval_metric': 'mlogloss', 'gamma': 0.7000000000000001, 'max_depth': 1, 'min_child_weight': 3.0, 'n_estimators': 128.0, 'nthread': 4, 'num_class': 9, 'objective': 'multi:softmax', 'seed': 314159265, 'silent': 1, 'subsample': 0.65, 'tree_method': 'exact'}
  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

c:\Users\jan67814\AppData\Local\miniforge3\envs\env_dl_ml\lib\site-packages\xgboost\core.py:158: UserWarning: [12:34:54] WARNING: D:\bld\xgboost-split_1738395203399\work\src\learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)



[0]	eval-mlogloss:1.85689	train-mlogloss:1.84919     
[1]	eval-mlogloss:1.71032	train-mlogloss:1.69242     
[2]	eval-mlogloss:1.60690	train-mlogloss:1.58130     
[3]	eval-mlogloss:1.53832	train-mlogloss:1.50558     
[4]	eval-mlogloss:1.48368	train-mlogloss:1.44821     
[5]	eval-mlogloss:1.44155	train-mlogloss:1.40150     
[6]	eval-mlogloss:1.40309	train-mlogloss:1.36350     
[7]	eval-mlogloss:1.37187	train-mlogloss:1.33125     
[8]	eval-mlogloss:1.34464	train-mlogloss:1.30272     
[9]	eval-mlogloss:1.31781	train-mlogloss:1.27850     
[10]	eval-mlogloss:1.29291	train-mlogloss:1.25566    
[11]	eval-mlogloss:1.27298	train-mlogloss:1.23539    
[12]	eval-mlogloss:1.25282	train-mlogloss:1.21562    
[13]	eval-mlogloss:1.23582	train-mlogloss:1.19838    
[14]	eval-mlogloss:1.22125	train-mlogloss:1.18275    
[15]	eval-mlogloss:1.20750	train-mlogloss:1.16811    
[16]	eval-mlogloss:1.19239	train-mlogloss:1.15402    
[17]	eval-mlogloss:1.17968	train-mlogloss:1.14080    
[18]	eval-mlogloss:1.16844	t

c:\Users\jan67814\AppData\Local\miniforge3\envs\env_dl_ml\lib\site-packages\xgboost\core.py:158: UserWarning: [12:35:21] WARNING: D:\bld\xgboost-split_1738395203399\work\src\learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)



[0]	eval-mlogloss:1.65931	train-mlogloss:1.64361                               
[1]	eval-mlogloss:1.39705	train-mlogloss:1.37359                               
[2]	eval-mlogloss:1.20732	train-mlogloss:1.17828                               
[3]	eval-mlogloss:1.06725	train-mlogloss:1.03522                               
[4]	eval-mlogloss:0.96116	train-mlogloss:0.92717                               
[5]	eval-mlogloss:0.87696	train-mlogloss:0.84164                               
[6]	eval-mlogloss:0.80126	train-mlogloss:0.76428                               
[7]	eval-mlogloss:0.74627	train-mlogloss:0.70888                               
[8]	eval-mlogloss:0.69670	train-mlogloss:0.65898                               
[9]	eval-mlogloss:0.65081	train-mlogloss:0.61229                               
[10]	eval-mlogloss:0.61519	train-mlogloss:0.57655                              
[11]	eval-mlogloss:0.57982	train-mlogloss:0.54165                              
[12]	eval-mlogloss:0.55293	train-mloglos

...

[0]	eval-mlogloss:1.60265	train-mlogloss:1.57353                                 
[1]	eval-mlogloss:1.29391	train-mlogloss:1.25246                                 
[2]	eval-mlogloss:1.06870	train-mlogloss:1.01882                                 
[3]	eval-mlogloss:0.90362	train-mlogloss:0.84986                                 
[4]	eval-mlogloss:0.77891	train-mlogloss:0.72272                                 
[5]	eval-mlogloss:0.67573	train-mlogloss:0.61853                                 
[6]	eval-mlogloss:0.58837	train-mlogloss:0.52970                                 
[7]	eval-mlogloss:0.51806	train-mlogloss:0.45900                                 
[8]	eval-mlogloss:0.45784	train-mlogloss:0.39916                                 
[9]	eval-mlogloss:0.40952	train-mlogloss:0.35223                                 
[10]	eval-mlogloss:0.36822	train-mlogloss:0.31131                                
[11]	eval-mlogloss:0.33087	train-mlogloss:0.27540                                
[12]	eval-mloglo

c:\Users\jan67814\AppData\Local\miniforge3\envs\env_dl_ml\lib\site-packages\xgboost\core.py:158: UserWarning: [12:39:36] WARNING: D:\bld\xgboost-split_1738395203399\work\src\learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)



[0]	eval-mlogloss:1.73513	train-mlogloss:1.72692                                  
[1]	eval-mlogloss:1.55372	train-mlogloss:1.53245                                  
[2]	eval-mlogloss:1.42851	train-mlogloss:1.40186                                  
[3]	eval-mlogloss:1.34232	train-mlogloss:1.30933                                  
[4]	eval-mlogloss:1.27912	train-mlogloss:1.24345                                  
[5]	eval-mlogloss:1.22944	train-mlogloss:1.19157                                  
[6]	eval-mlogloss:1.18511	train-mlogloss:1.14705                                  
[7]	eval-mlogloss:1.14997	train-mlogloss:1.11244                                  
[8]	eval-mlogloss:1.11988	train-mlogloss:1.08388                                  
[9]	eval-mlogloss:1.09456	train-mlogloss:1.05817                                  
[10]	eval-mlogloss:1.07097	train-mlogloss:1.03483                                 
[11]	eval-mlogloss:1.04994	train-mlogloss:1.01324                                 
[12]

c:\Users\jan67814\AppData\Local\miniforge3\envs\env_dl_ml\lib\site-packages\xgboost\core.py:158: UserWarning: [12:40:18] WARNING: D:\bld\xgboost-split_1738395203399\work\src\learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)



[0]	eval-mlogloss:1.51758	train-mlogloss:1.49303                                 
[1]	eval-mlogloss:1.21468	train-mlogloss:1.17873                                 
[2]	eval-mlogloss:1.00941	train-mlogloss:0.96655                                 
[3]	eval-mlogloss:0.85791	train-mlogloss:0.81151                                 
[4]	eval-mlogloss:0.74405	train-mlogloss:0.69545                                 
[5]	eval-mlogloss:0.64906	train-mlogloss:0.59925                                 
[6]	eval-mlogloss:0.57379	train-mlogloss:0.52398                                 
[7]	eval-mlogloss:0.51436	train-mlogloss:0.46511                                 
[8]	eval-mlogloss:0.46688	train-mlogloss:0.41822                                 
[9]	eval-mlogloss:0.42707	train-mlogloss:0.37923                                 
[10]	eval-mlogloss:0.39612	train-mlogloss:0.34839                                
[11]	eval-mlogloss:0.36837	train-mlogloss:0.32040                                
[12]	eval-mloglo

In [9]:
# Train and test a classifier
def train_and_test(X_tr, y_tr, X_v, well_v):
    
    # Feature normalization
    scaler = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_tr)
    X_tr = scaler.transform(X_tr)
    X_v = scaler.transform(X_v)
    
    # Train classifier
    #clf = make_pipeline(make_union(VotingClassifier([("est", ExtraTreesClassifier(criterion="gini", max_features=1.0, n_estimators=500))]), FunctionTransformer(lambda X: X)), XGBClassifier(learning_rate=0.73, max_depth=10, min_child_weight=10, n_estimators=500, subsample=0.27))
    #clf =  make_pipeline( KNeighborsClassifier(n_neighbors=5, weights="distance") ) 
    #clf = make_pipeline(MaxAbsScaler(),make_union(VotingClassifier([("est", RandomForestClassifier(n_estimators=500))]), FunctionTransformer(lambda X: X)),ExtraTreesClassifier(criterion="entropy", max_features=0.0001, n_estimators=500))
    # * clf = make_pipeline( make_union(VotingClassifier([("est", BernoulliNB(alpha=60.0, binarize=0.26, fit_prior=True))]), FunctionTransformer(lambda X: X)),RandomForestClassifier(n_estimators=500))
    # ** clf = make_pipeline ( XGBClassifier(learning_rate=0.12, max_depth=3, min_child_weight=10, n_estimators=150, seed = 17, colsample_bytree = 0.9) )
    clf = make_pipeline ( XGBClassifier(learning_rate=0.15, max_depth=8, min_child_weight=4, n_estimators=148, seed = SEED, colsample_bytree = 0.85, subsample = 0.9 , gamma = 0.75) )
    clf.fit(X_tr, y_tr)
    
    # Test classifier
    y_v_hat = clf.predict(X_v)
    
    # Clean isolated facies for each well
    for w in np.unique(well_v):
        y_v_hat[well_v==w] = medfilt(y_v_hat[well_v==w], kernel_size=5)
    
    return y_v_hat

## Prediction

In [10]:

#Load testing data
test_data = pd.read_csv('../validation_data_nofacies.csv')

# Prepare training data
X_tr = X
y_tr = y

# Augment features
X_tr, padded_rows = augment_features(X_tr, well, depth)

# Removed padded rows
X_tr = np.delete(X_tr, padded_rows, axis=0)
y_tr = np.delete(y_tr, padded_rows, axis=0) - 1

# Prepare test data
well_ts = test_data['Well Name'].values
depth_ts = test_data['Depth'].values
X_ts = test_data[feature_names].values

# Augment features
X_ts, padded_rows = augment_features(X_ts, well_ts, depth_ts)

# Predict test labels
y_ts_hat = train_and_test(X_tr, y_tr, X_ts, well_ts)

# Save predicted labels
test_data['Facies'] = y_ts_hat + 1
test_data.to_csv('Prediction_XXX_Final.csv')